# Data Exploration

In [1]:
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import re
import scipy.stats as stats

from autoimmune_foundation.dataset import main


2025-03-10 15:56:56.587 | INFO     | autoimmune_foundation.config:<module>:11 - PROJ_ROOT path is: /Users/joaofelipe/Yale/Autoimmune Foundation/4_Code/autoimmune_foundation


# LOG2

## Creating the Protein x Patients DataFrame

In [34]:
# We should use the RED data
huprot_raw_df = pd.read_csv('../data/raw/LLD_vs_H_RawLog2MFI_635nm_red.csv',encoding='latin-1')
huprot_raw_df.columns = [col.split("_")[-1] for col in huprot_raw_df.columns]
huprot_raw_df = huprot_raw_df.drop(['Block','Column','Rows','Name'],axis=1)
huprot_raw_df = huprot_raw_df[huprot_raw_df['ID'] != 'ND']
huprot_raw_df = huprot_raw_df[huprot_raw_df['ID'] != 'Control']
huprot_raw_df = huprot_raw_df.drop(['IgG','gst'],axis=1)

In [35]:
huprot_raw_df

,ID,P092,P094,P099,P102,P104,P110,P121,P125,P126,...,P106,P115,P116,P120,P127,P130,P133,P134,P138,P148
0,JHU00260.B1C1R2,5.8074,5.8455,5.7142,5.5999,5.6439,5.8948,5.7142,5.5850,5.7682,...,5.2479,4.7814,5.0224,4.8074,5.0444,5.0224,5.0444,4.9307,5.3923,5.3038
1,JHU00055.B1C2R2,5.9425,6.2668,5.7279,5.7004,5.7682,5.8580,5.8202,5.6439,5.9542,...,5.3750,5.1699,4.9773,4.8580,5.0875,5.3038,5.2854,5.1085,6.2854,5.7814
2,JHU00028.B1C3R2,5.9307,6.0112,5.8329,5.8704,5.9069,5.8329,5.6865,5.5850,5.9887,...,5.2854,5.0224,5.1085,4.8826,5.0000,5.2288,5.1898,5.1497,5.3750,5.7682
4,JHU00058.B1C5R2,8.0224,8.6457,7.2574,6.9484,7.1849,7.2046,7.6147,6.2668,7.5736,...,7.2761,7.4512,7.0389,6.1699,7.0389,6.5925,7.6366,6.3487,10.9480,8.0389
5,JHU00220.B1C6R2,5.8074,5.8580,5.7944,5.8074,5.7682,5.8826,5.6724,5.5546,5.8202,...,5.3038,5.0661,4.9069,4.9307,4.9307,4.9773,5.1293,5.2095,5.4594,5.5236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26326,JHU30334.B20C23R42,5.6294,5.7004,5.5699,5.5699,5.4919,5.6582,5.7004,5.5546,5.9069,...,5.2854,5.0224,4.9773,4.9773,5.0224,5.0444,5.2288,5.1699,5.3399,5.2479
26329,JHU30325.B20C26R42,5.5999,5.7682,5.6439,5.4757,5.5699,5.5850,5.5699,5.5850,5.9425,...,5.1085,5.2095,5.9658,5.0000,4.8580,5.1699,5.1085,4.9773,5.2095,5.1293
26330,JHU30413.B20C27R42,5.5078,5.6865,5.7682,5.6147,5.5546,5.5699,5.7415,5.5850,5.8704,...,5.3038,5.0444,4.8580,4.8329,4.8074,5.0661,5.0444,5.1085,5.2479,5.1699
26332,JHU30444.B20C29R42,5.7279,5.8329,5.5546,5.6582,5.6724,5.5699,5.5850,5.5078,5.9069,...,5.3399,5.1497,5.0875,4.8074,4.9773,5.1699,5.2668,4.8580,5.2479,5.2854


In [36]:
patient_class_df = pd.read_csv('../data/raw/studyManifest_LLD_vs_H.csv',encoding='latin-1')
patient_class_df.head(1)

,sourceName,studyName,studyGroup
0,CDI9091876_P092_01242025_0001,P092,Healthy


## Creating the Patients x Proteins

In [37]:
# Add the class row
class_dict = dict(zip(patient_class_df["studyName"], patient_class_df["studyGroup"]))
# huprot_raw_df.loc["studyGroup"] = huprot_raw_df.columns.map(class_dict)

huprot_raw_df

,ID,P092,P094,P099,P102,P104,P110,P121,P125,P126,...,P106,P115,P116,P120,P127,P130,P133,P134,P138,P148
0,JHU00260.B1C1R2,5.8074,5.8455,5.7142,5.5999,5.6439,5.8948,5.7142,5.5850,5.7682,...,5.2479,4.7814,5.0224,4.8074,5.0444,5.0224,5.0444,4.9307,5.3923,5.3038
1,JHU00055.B1C2R2,5.9425,6.2668,5.7279,5.7004,5.7682,5.8580,5.8202,5.6439,5.9542,...,5.3750,5.1699,4.9773,4.8580,5.0875,5.3038,5.2854,5.1085,6.2854,5.7814
2,JHU00028.B1C3R2,5.9307,6.0112,5.8329,5.8704,5.9069,5.8329,5.6865,5.5850,5.9887,...,5.2854,5.0224,5.1085,4.8826,5.0000,5.2288,5.1898,5.1497,5.3750,5.7682
4,JHU00058.B1C5R2,8.0224,8.6457,7.2574,6.9484,7.1849,7.2046,7.6147,6.2668,7.5736,...,7.2761,7.4512,7.0389,6.1699,7.0389,6.5925,7.6366,6.3487,10.9480,8.0389
5,JHU00220.B1C6R2,5.8074,5.8580,5.7944,5.8074,5.7682,5.8826,5.6724,5.5546,5.8202,...,5.3038,5.0661,4.9069,4.9307,4.9307,4.9773,5.1293,5.2095,5.4594,5.5236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26326,JHU30334.B20C23R42,5.6294,5.7004,5.5699,5.5699,5.4919,5.6582,5.7004,5.5546,5.9069,...,5.2854,5.0224,4.9773,4.9773,5.0224,5.0444,5.2288,5.1699,5.3399,5.2479
26329,JHU30325.B20C26R42,5.5999,5.7682,5.6439,5.4757,5.5699,5.5850,5.5699,5.5850,5.9425,...,5.1085,5.2095,5.9658,5.0000,4.8580,5.1699,5.1085,4.9773,5.2095,5.1293
26330,JHU30413.B20C27R42,5.5078,5.6865,5.7682,5.6147,5.5546,5.5699,5.7415,5.5850,5.8704,...,5.3038,5.0444,4.8580,4.8329,4.8074,5.0661,5.0444,5.1085,5.2479,5.1699
26332,JHU30444.B20C29R42,5.7279,5.8329,5.5546,5.6582,5.6724,5.5699,5.5850,5.5078,5.9069,...,5.3399,5.1497,5.0875,4.8074,4.9773,5.1699,5.2668,4.8580,5.2479,5.2854


In [38]:
patients_huprot_raw = huprot_raw_df.set_index('ID').transpose()
patients_huprot_raw["ID"] = patients_huprot_raw.index
class_dict = dict(zip(patient_class_df["studyName"], patient_class_df["studyGroup"]))

# Map patient classes using their IDs
patients_huprot_raw["patients_class"] = patients_huprot_raw["ID"].map(class_dict)

patients_huprot_raw = patients_huprot_raw.reset_index(drop=True)

patients_huprot_raw

ID,JHU00260.B1C1R2,JHU00055.B1C2R2,JHU00028.B1C3R2,JHU00058.B1C5R2,JHU00220.B1C6R2,JHU00167.B1C8R2,JHU00110.B1C9R2,JHU00370.B1C10R2,JHU00250.B1C11R2,JHU00048.B1C12R2,...,JHU30452.B20C17R42,JHU30401.B20C20R42,JHU30450.B20C22R42,JHU30334.B20C23R42,JHU30325.B20C26R42,JHU30413.B20C27R42,JHU30444.B20C29R42,JHU30411.B20C32R42,ID,patients_class
0,5.8074,5.9425,5.9307,8.0224,5.8074,5.8202,6.8580,6.1189,5.8329,5.7549,...,5.5699,5.6294,5.5699,5.6294,5.5999,5.5078,5.7279,5.5850,P092,Healthy
1,5.8455,6.2668,6.0112,8.6457,5.8580,6.0112,6.4429,6.4429,6.0334,5.8704,...,5.5999,5.6582,5.7142,5.7004,5.7682,5.6865,5.8329,5.7682,P094,Healthy
2,5.7142,5.7279,5.8329,7.2574,5.7944,5.6439,5.8329,5.7279,5.6439,5.7814,...,5.6147,5.7004,5.5392,5.5699,5.6439,5.7682,5.5546,5.8074,P099,Healthy
3,5.5999,5.7004,5.8704,6.9484,5.8074,5.5850,5.7549,5.6724,5.6439,5.7004,...,5.5546,5.5850,5.6439,5.5699,5.4757,5.6147,5.6582,5.7142,P102,Healthy
4,5.6439,5.7682,5.9069,7.1849,5.7682,5.7944,5.9542,5.7279,5.5850,5.7944,...,5.6294,5.6724,5.6865,5.4919,5.5699,5.5546,5.6724,5.6724,P104,Healthy
5,5.8948,5.8580,5.8329,7.2046,5.8826,5.7415,6.0875,5.8455,5.8704,5.7279,...,5.5236,5.7142,5.5546,5.6582,5.5850,5.5699,5.5699,5.6865,P110,Healthy
6,5.7142,5.8202,5.6865,7.6147,5.6724,5.5699,6.0000,5.7682,5.7415,5.6439,...,5.6724,5.6147,5.7279,5.7004,5.5699,5.7415,5.5850,5.7682,P121,Healthy
7,5.5850,5.6439,5.5850,6.2668,5.5546,5.6294,6.1189,8.3619,5.5392,5.5699,...,5.5699,5.5392,5.6294,5.5546,5.5850,5.5850,5.5078,5.5236,P125,Healthy
8,5.7682,5.9542,5.9887,7.5736,5.8202,5.7944,6.7415,5.8580,5.9542,5.8948,...,5.9307,5.8455,5.8704,5.9069,5.9425,5.8704,5.9069,6.0224,P126,Healthy
9,5.4757,5.6439,5.5392,7.7381,5.5999,5.6439,5.6582,5.7944,5.5392,5.4919,...,5.5699,5.6294,5.6294,5.7415,5.5699,5.6582,5.5392,5.6147,P129,Healthy


In [39]:
patients_huprot_raw.to_csv('../data/processed/patients_proteins_log2.csv')